In [1]:
import pandas as pd
import tabula

In [2]:
class Person:
    def __init__(self, nome, mensalidade):
        self.nome = nome
        self.mensalidade = mensalidade
        self.dependencia = None
        self.data_nascimento = None
        self.data_inclusao = None
        self.dependentes = []
        self.utilizacao = 0
        self.taxa_boleto = 2.93
        self.reaj_retro = 0
        self.reaj_faixa = 0
        
    def get_valor_mensalidades(self):
        valor_dependentes = sum([x.mensalidade for x in self.dependentes])
        return self.mensalidade + valor_dependentes
    
    def get_valor_utilizacao(self):
        valor_dependentes = sum(x.utilizacao for x in self.dependentes)
        return self.utilizacao + valor_dependentes
    
    def get_valor_reaj_retroativo(self):
        valor_dependentes = sum([x.reaj_retro for x in self.dependentes])
        return self.reaj_retro + valor_dependentes
    
    def get_valor_reaj_faixa(self):
        valor_dependentes = sum([x.reaj_faixa for x in self.dependentes])
        return self.reaj_faixa + valor_dependentes
    
    def get_valor_total(self):
        return self.get_valor_mensalidades() + self.get_valor_utilizacao() + self.taxa_boleto + self.get_valor_reaj_retroativo() + self.get_valor_reaj_faixa()    
    
    def __str__(self):
        return f'{self.nome} - {self.get_valor_total()}'
    
    def __repr__(self):
        return f'{self.nome} - {self.get_valor_total()}'
    
    def report_values(self):
        l = []
        l.append(f'{self.nome}: {self.mensalidade:.2f}')
        l.append(f'Utilização: {self.utilizacao:.2f}')
        l.append(f'Reajuste retroativo: {self.reaj_retro:.2f}')
        l.append(f'Reajuste faixa etária: {self.reaj_faixa:.2f}')
        for p in self.dependentes:
            l.append(f'{p.nome}: {p.mensalidade:.2f}')
            l.append(f'Reajuste retroativo: {p.reaj_retro:.2f}')
            l.append(f'Reajuste faixa etária: {p.reaj_faixa:.2f}')
        return l
    
    

In [3]:
p = Person('Juliano Fischer Naves',400)
p1 = Person('Jhaneffer',300)
p2 = Person('Juliane',200)
p3 = Person('Olívia',100)
l = [p1, p2, p3]
p.dependentes.extend(l)

In [4]:
p.get_valor_total()

1002.93

In [5]:
p.mensalidade

400

In [6]:
FILE_REPORT_CONTRATOS = "faturamentoagosto21/Relatório de Mensalidades 67.pdf"
FILE_REPORT_USO = "faturamentoagosto21/Relatório de Utilização 67.pdf"

In [7]:
df = tabula.read_pdf(FILE_REPORT_CONTRATOS, pages='all')

In [8]:
df[0].head()

,Lotação,Beneficiário,Nome,Plano,Dependência,Dt. Nascimento,Dt. Inclusão,Mensalidade,Reaj.\rRetroativo,Reaj.\rFaixa,Total,C/Corrente
0,-,67000001004,Christiane Souza De\rMacedo Barbosa,Empr Nac Esp\rC/Cop,Titulares,09/08/1985,25/07/2014,"389,24","0,00","0,00","389,24",NaN
1,-,67000001012,Max Pederiva Barbosa,Empr Nac Esp\rC/Cop,Companheiro (a),10/06/1984,25/07/2014,"389,24","0,00","0,00","389,24",NaN
2,-,67000001101,Igor Souza Pederiva,Empr Nac Esp\rC/Cop,filhos (as),13/12/2016,05/01/2017,"176,97","0,00","0,00","176,97",NaN
3,-,67000005000,Melquisedeque Da\rConceicao Lima,Empr Nac Esp\rC/Cop,Titulares,22/06/1984,25/07/2014,"389,24","0,00","0,00","389,24",NaN
4,-,67000005107,Sofia Folle Lima,Empr Nac Esp\rC/Cop,filhos (as),24/01/2014,25/07/2014,"176,97","0,00","0,00","176,97",NaN


In [9]:
df[13]

,Lotação,Beneficiário,Nome,Plano,Dependência,Dt. Nascimento,Dt. Inclusão,Mensalidade,Reaj.\rRetroativo,Reaj.\rFaixa,Total,C/Corrente
0,-,0067000103109,Otto Murilo Alves\rFonseca,Empr Nac Esp\rC/Cop,filhos (as),27/06/2006,13/02/2020,"176,97","0,00","0,00","176,97",NaN
1,-,0067000103117,Rebeca Alves Fonseca,Empr Nac Esp\rC/Cop,filhos (as),31/03/2009,13/02/2020,"176,97","0,00","0,00","176,97",NaN
2,-,0067000105004,Elza Moreira Alves,Empr Nac Esp\rC/Cop,Titulares,11/08/1976,05/06/2020,"488,05","0,00","9,70","497,75",NaN
3,-,0067000105101,Sofia Aurea Alves\rRibeiro,Empr Nac Esp\rC/Cop,filhos (as),15/09/2016,05/06/2020,"176,97","0,00","0,00","176,97",NaN
4,-,0067000106000,Jefferson Castro\rCasseano Furtado,Empr Nac Esp\rC/Cop,Titulares,03/02/1990,10/08/2020,"338,46","0,00","0,00","338,46",NaN
5,-,0067000106019,Andreza Furtado\rGoncalves Castro,Empr Nac Esp\rC/Cop,Companheiro (a),05/06/1992,10/08/2020,"338,46","0,00","0,00","338,46",NaN
6,-,0067000107007,Aurea Dayse Cosmo Da\rSilva,Empr Nac Esp\rC/Cop,Titulares,17/04/1990,24/08/2020,"338,46","0,00","0,00","338,46",NaN
7,-,0067000108003,Ricardo Vanjura Ferreira,Empr Nac Esp\rC/Cop,Titulares,07/09/1988,26/07/2021,"338,46","0,00","0,00","338,46",NaN
8,-,0067000108011,Ana Paula Grabovski,Empr Nac Esp\rC/Cop,Companheiro (a),11/01/1992,26/07/2021,"338,46","0,00","0,00","338,46",NaN
9,-,"69.128,37","0,00","206,18","69.334,55",-,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#df[13]
df[13].drop([9,10], inplace=True)
#df[13]
#df[8] tá zikado
#df[1].drop('C/Corrente',axis=1)

In [11]:
df[13]

,Lotação,Beneficiário,Nome,Plano,Dependência,Dt. Nascimento,Dt. Inclusão,Mensalidade,Reaj.\rRetroativo,Reaj.\rFaixa,Total,C/Corrente
0,-,0067000103109,Otto Murilo Alves\rFonseca,Empr Nac Esp\rC/Cop,filhos (as),27/06/2006,13/02/2020,"176,97","0,00","0,00","176,97",NaN
1,-,0067000103117,Rebeca Alves Fonseca,Empr Nac Esp\rC/Cop,filhos (as),31/03/2009,13/02/2020,"176,97","0,00","0,00","176,97",NaN
2,-,0067000105004,Elza Moreira Alves,Empr Nac Esp\rC/Cop,Titulares,11/08/1976,05/06/2020,"488,05","0,00","9,70","497,75",NaN
3,-,0067000105101,Sofia Aurea Alves\rRibeiro,Empr Nac Esp\rC/Cop,filhos (as),15/09/2016,05/06/2020,"176,97","0,00","0,00","176,97",NaN
4,-,0067000106000,Jefferson Castro\rCasseano Furtado,Empr Nac Esp\rC/Cop,Titulares,03/02/1990,10/08/2020,"338,46","0,00","0,00","338,46",NaN
5,-,0067000106019,Andreza Furtado\rGoncalves Castro,Empr Nac Esp\rC/Cop,Companheiro (a),05/06/1992,10/08/2020,"338,46","0,00","0,00","338,46",NaN
6,-,0067000107007,Aurea Dayse Cosmo Da\rSilva,Empr Nac Esp\rC/Cop,Titulares,17/04/1990,24/08/2020,"338,46","0,00","0,00","338,46",NaN
7,-,0067000108003,Ricardo Vanjura Ferreira,Empr Nac Esp\rC/Cop,Titulares,07/09/1988,26/07/2021,"338,46","0,00","0,00","338,46",NaN
8,-,0067000108011,Ana Paula Grabovski,Empr Nac Esp\rC/Cop,Companheiro (a),11/01/1992,26/07/2021,"338,46","0,00","0,00","338,46",NaN


In [12]:
def df_filter_procedure(df):
    #df = df.drop(['Unnamed: 7', 'Unnamed: 2'], axis=1)
    #df = df.drop(df.columns[-1],axis=1)
    df = df.drop(['Plano', 'Total', 'C/Corrente','Lotação'], axis=1)
    df = df.dropna()
    #df.drop(df.index[0], inplace=True)
    values = ['contrato', 'nome', 'dependencia', 'data_nascimento', 'data_inclusao', 'mensalidade', 'reajuste_retroativo', 'reajuste_faixa']
    keys = ['Beneficiário', 'Nome', 'Dependência', 'Dt. Nascimento', 'Dt. Inclusão', 'Mensalidade','Reaj.\rRetroativo','Reaj.\rFaixa']
    d = dict(zip(keys, values))
    #df['Unnamed: 1'] = df['Unnamed: 1'].apply(lambda x: x.split(" Nac")[0])
    # df = df.drop(['nome','plano'],axis=1)
    df.rename(columns=d, inplace=True)
    return df

In [13]:
pages = []
#ignora a última porque o df[8] tá zikado
#for page in df[:-1]:
for page in df:
    pages.append(page)
pages = pd.concat(pages)

In [14]:
#até aqui tudo bem... arrumar um jeito de filtrar o df[8]
#df[8]

In [15]:
#descarta as duas primeiras linhas e a última
'''df[8].drop([0,1,19], inplace=True)
df[8].drop(['Unnamed: 7', 'Unnamed: 4'], axis=1, inplace=True)
df[8]['Reaj.'] = df[8]['Reaj.'].apply(lambda x: x.split(" ")[0])
df[8]['Unnamed: 2'] = df[8]['Unnamed: 2'].apply(lambda x: x.split('C/Cop ')[1])
values = ['contrato', 'nome', 'dependencia', 'data_nascimento', 'data_inclusao', 'mensalidade', 'reajuste_retroativo']
keys = ['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 6','Reaj.']
d = dict(zip(keys, values))
df[8].rename(columns=d, inplace=True)
df[8]'''
#values = ['contrato', 'nome', 'dependencia', 'data_nascimento', 'data_inclusao', 'mensalidade', 'reajuste_retroativo']
#keys = ['Unnamed: 0','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 6','Reaj.']
#d = dict(zip(keys, values))
#df[8].rename(columns=d, inplace=True)
#df[8].head()

'df[8].drop([0,1,19], inplace=True)\ndf[8].drop([\'Unnamed: 7\', \'Unnamed: 4\'], axis=1, inplace=True)\ndf[8][\'Reaj.\'] = df[8][\'Reaj.\'].apply(lambda x: x.split(" ")[0])\ndf[8][\'Unnamed: 2\'] = df[8][\'Unnamed: 2\'].apply(lambda x: x.split(\'C/Cop \')[1])\nvalues = [\'contrato\', \'nome\', \'dependencia\', \'data_nascimento\', \'data_inclusao\', \'mensalidade\', \'reajuste_retroativo\']\nkeys = [\'Unnamed: 0\',\'Unnamed: 1\',\'Unnamed: 2\',\'Unnamed: 3\',\'Unnamed: 5\',\'Unnamed: 6\',\'Reaj.\']\nd = dict(zip(keys, values))\ndf[8].rename(columns=d, inplace=True)\ndf[8]'

In [16]:
pages = df_filter_procedure(pages)
#pages = pd.concat([pages, df[8]])
pages.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)
pages.head()

,contrato,nome,dependencia,data_nascimento,data_inclusao,mensalidade,reajuste_retroativo,reajuste_faixa
0,67000001004,Christiane Souza De Macedo Barbosa,Titulares,09/08/1985,25/07/2014,"389,24","0,00","0,00"
1,67000001012,Max Pederiva Barbosa,Companheiro (a),10/06/1984,25/07/2014,"389,24","0,00","0,00"
2,67000001101,Igor Souza Pederiva,filhos (as),13/12/2016,05/01/2017,"176,97","0,00","0,00"
3,67000005000,Melquisedeque Da Conceicao Lima,Titulares,22/06/1984,25/07/2014,"389,24","0,00","0,00"
4,67000005107,Sofia Folle Lima,filhos (as),24/01/2014,25/07/2014,"176,97","0,00","0,00"


In [17]:
def convert_coin_(x):
    x = x.replace('.','')
    x = x.replace(',','.')
    return x
pages['mensalidade'] = pages['mensalidade'].apply(lambda x: convert_coin_(x))
#AttributeError: 'float' object has no attribute 'replace'

In [18]:
pages['reajuste_retroativo'] = pages['reajuste_retroativo'].apply(lambda x: convert_coin_(x))
pages['reajuste_faixa'] = pages['reajuste_faixa'].apply(lambda x: convert_coin_(x))
#pages
#AttributeError: 'float' object has no attribute 'replace'

In [19]:
pages

,contrato,nome,dependencia,data_nascimento,data_inclusao,mensalidade,reajuste_retroativo,reajuste_faixa
0,67000001004,Christiane Souza De Macedo Barbosa,Titulares,09/08/1985,25/07/2014,389.24,0.00,0.00
1,67000001012,Max Pederiva Barbosa,Companheiro (a),10/06/1984,25/07/2014,389.24,0.00,0.00
2,67000001101,Igor Souza Pederiva,filhos (as),13/12/2016,05/01/2017,176.97,0.00,0.00
3,67000005000,Melquisedeque Da Conceicao Lima,Titulares,22/06/1984,25/07/2014,389.24,0.00,0.00
4,67000005107,Sofia Folle Lima,filhos (as),24/01/2014,25/07/2014,176.97,0.00,0.00
...,...,...,...,...,...,...,...,...
4,0067000106000,Jefferson Castro Casseano Furtado,Titulares,03/02/1990,10/08/2020,338.46,0.00,0.00
5,0067000106019,Andreza Furtado Goncalves Castro,Companheiro (a),05/06/1992,10/08/2020,338.46,0.00,0.00
6,0067000107007,Aurea Dayse Cosmo Da Silva,Titulares,17/04/1990,24/08/2020,338.46,0.00,0.00
7,0067000108003,Ricardo Vanjura Ferreira,Titulares,07/09/1988,26/07/2021,338.46,0.00,0.00


In [20]:
titulares = []
titular_atual = None
pessoas_set = set()

for index, row in pages.iterrows():
    mensalidade = float(row[5])
    p = Person(row[1], mensalidade) 
    p.dependencia = row[2]
    p.data_nascimento = row[3]
    p.data_inclusao = row[4]
    p.reaj_retro = float(row[6])
    p.reaj_faixa = float(row[7])
    if p.dependencia == 'Titulares':
        titulares.append(p)
        titular_atual = p
    else:
        titular_atual.dependentes.append(p)
    pessoas_set.add(p)

In [21]:
titulares

[Christiane Souza De Macedo Barbosa - 958.38,
 Melquisedeque Da Conceicao Lima - 569.14,
 Jaqueline Aida Ferrete - 667.9499999999999,
 Anelize Irene Appelt - 907.6,
 Aline Costa Elhke - 1033.79,
 Aremilson Elias De Oliveira - 785.26,
 Claudinei De Oliveira Pinho - 1976.5000000000002,
 Claudia Aparecida Prates Dos Santos - 1434.49,
 Maria Aparecida Boaventura - 841.0,
 Douglas Legramante - 907.6,
 Carlos Pereira Soares - 448.26,
 Julio Cezar Mozer Sodre - 2493.5499999999997,
 Antonio Sergio Florindo Dos Santos - 1455.38,
 Maria Consuelo Moreira - 942.78,
 Maria De Araujo Dal Moro - 680.8799999999999,
 Lirian Keli Dos Santos - 1208.42,
 Pedro Vargas Groeff - 2100.47,
 Ezequiel Ferreira Barbosa - 1536.1900000000003,
 Cristiano Perucchi - 1084.57,
 Ana Paula Schimidt Quadros - 569.14,
 Alvino Moraes De Amorim - 1882.6200000000001,
 Renato Delmonico - 1191.44,
 Gleiser Rodrigues De Melo - 1341.2,
 Cinthia Alves Pereira Mattos - 958.38,
 Aline Alves De Moraes - 1135.3500000000001,
 Sandra Ap

In [47]:
#Extrai somente a página 1
area  = [93.426,99.023,818.448,343.567]
#ut = tabula.read_pdf(FILE_REPORT_USO, pages='all', lattice=True, multiple_tables=False)
ut = tabula.read_pdf(FILE_REPORT_USO, pages=1, stream=True, multiple_tables=True, area=area, pandas_options={'header':None})
ut[0]

,0,1
0,Melquisedeque Da Conceicao Lima,"R$ 198,22"
1,Jaqueline Aida Ferrete,"R$ 81,99"
2,Anelize Irene Appelt,"R$ 273,15"
3,Aline Costa Elhke,"R$ 558,62"
4,Aremilson Elias De Oliveira,"R$ 72,22"
5,Claudinei De Oliveira Pinho,"R$ 34,30"
6,Claudia Aparecida Prates Dos Santos,"R$ 300,19"
7,Maria Aparecida Boaventura,"R$ 139,50"
8,Douglas Legramante,"R$ 143,56"
9,Julio Cezar Mozer Sodre,"R$ 19,05"


In [79]:
# segunda página
temp = pd.DataFrame([['Leticia Alves Fonseca', 'R$ 81,92'],
              ['Elza Moreira Alves', 'R$ 34,31'],
              ['Jefferson Castro Casseano Furtado', 'R$ 30,87'], 
              ['Aurea Dayse Cosmo Da Silva', 'R$ 103,81']])
ut[0] = pd.concat([ut[0], temp], ignore_index=True)
ut[0]

,0,1
0,Melquisedeque Da Conceicao Lima,"R$ 198,22"
1,Jaqueline Aida Ferrete,"R$ 81,99"
2,Anelize Irene Appelt,"R$ 273,15"
3,Aline Costa Elhke,"R$ 558,62"
4,Aremilson Elias De Oliveira,"R$ 72,22"
5,Claudinei De Oliveira Pinho,"R$ 34,30"
6,Claudia Aparecida Prates Dos Santos,"R$ 300,19"
7,Maria Aparecida Boaventura,"R$ 139,50"
8,Douglas Legramante,"R$ 143,56"
9,Julio Cezar Mozer Sodre,"R$ 19,05"


In [57]:
#esta nao precisou em maio
#area_2 = [75.311,100.745,120.087,338.401]
#Extrai somente a página 2
#ut = tabula.read_pdf(FILE_REPORT_USO, pages='all', lattice=True, multiple_tables=False)
#ut2 = tabula.read_pdf(FILE_REPORT_USO, pages=2, stream=True, multiple_tables=False, area=area_2, pandas_options={'header':None})
#ut2[0]

In [58]:
'''def replace_name(wrong_name, list_of_correct_names):
    for name in list_of_correct_names:
        if wrong_name in name:
            return name
        
nomes_corretos = [t.nome for t in titulares]
nomes_corretos.append("Ney Grequi Franco Figueiredo")
nomes_corretos.append("Alexandre Vieira Saboia")
nomes_corretos.append("Claudemir Miranda Barboza")'''

'def replace_name(wrong_name, list_of_correct_names):\n    for name in list_of_correct_names:\n        if wrong_name in name:\n            return name\n        \nnomes_corretos = [t.nome for t in titulares]\nnomes_corretos.append("Ney Grequi Franco Figueiredo")\nnomes_corretos.append("Alexandre Vieira Saboia")\nnomes_corretos.append("Claudemir Miranda Barboza")'

In [86]:
ut0 = ut[0]
dict_ut = {}
for index, row in ut0.iterrows():
    #mensalidade = float(row[2][3:].replace(',','.'))
    #mensalidade = row[2]
    mensalidade = row[1]
    if mensalidade != 'Valor':
        mensalidade = float(row[1].replace(',','.').replace("R$ ",''))
        nome=row[0]
        dict_ut[nome] = mensalidade
        print(nome + " " + str(mensalidade))
    else:
        continue
sum(dict_ut.values())

Melquisedeque Da Conceicao Lima 198.22
Jaqueline Aida Ferrete 81.99
Anelize Irene Appelt 273.15
Aline Costa Elhke 558.62
Aremilson Elias De Oliveira 72.22
Claudinei De Oliveira Pinho 34.3
Claudia Aparecida Prates Dos Santos 300.19
Maria Aparecida Boaventura 139.5
Douglas Legramante 143.56
Julio Cezar Mozer Sodre 19.05
Antonio Sergio Florindo Dos Santos 256.83
Maria Consuelo Moreira 307.79
Maria De Araujo Dal Moro 219.0
Lirian Keli Dos Santos 98.01
Pedro Vargas Groeff 115.63
Ezequiel Ferreira Barbosa 30.87
Cristiano Perucchi 472.6
Ana Paula Schimidt Quadros 217.7
Alvino Moraes De Amorim 118.96
Renato Delmonico 117.15
Gleiser Rodrigues De Melo 90.0
Cinthia Alves Pereira Mattos 90.0
Aline Alves De Moraes 90.0
Alex Tavares De Almeida 54.66
Michel Osmar Costa Paiva 152.02
Agnaldo Correa De Almeida 86.94
Erlaine Correa Da Silva 90.0
Vera Lucia Ribeiro Azevedo 169.88
Vanuza De Paula Siqueira 65.0
Matusalem Aliares Da Silva 335.66
Marcio Marinho Martins 144.16
Aline Santiago 196.85
Rodrigo Ale

7752.180000000002

In [87]:
#não utilizada em maio
'''
ut2 = ut2[0]
dict_ut2 = {}
for index, row in ut2.iterrows():
    mensalidade = float(row[1][3:].replace(',','.'))
    #nome = replace_name(row[0], nomes_corretos)
    nome=row[0]
    dict_ut2[nome] = mensalidade
    print(nome)
sum(dict_ut2.values())'''

"\nut2 = ut2[0]\ndict_ut2 = {}\nfor index, row in ut2.iterrows():\n    mensalidade = float(row[1][3:].replace(',','.'))\n    #nome = replace_name(row[0], nomes_corretos)\n    nome=row[0]\n    dict_ut2[nome] = mensalidade\n    print(nome)\nsum(dict_ut2.values())"

In [88]:
#dict_ut2

In [89]:
#dict_ut = {**dict_ut, **dict_ut2} #merge two dicts
checked = set()
#for titular in titulares:
#    if titular.nome in dict_ut:
#        titular.utilizacao = dict_ut[titular.nome]
#        checked.add(titular.nome)
for pessoa in pessoas_set:
    if pessoa.nome in dict_ut:
        pessoa.utilizacao = dict_ut[pessoa.nome]
        checked.add(pessoa.nome)

In [90]:
# Encontra aqueles que não possuem mensalidade (desligados do plano), mas que 
# precisam pagar alguma utilização pendente
unchecked = set([t for t in dict_ut]) - checked
unchecked

set()

In [91]:
def computa_total_mensalidades(titulares):
    #return sum([t.get_valor_total() for t in titulares])
    return sum([t.get_valor_mensalidades() for t in titulares])

In [92]:
def computa_total_utilizacao(titulares):
    return sum([t.utilizacao for t in titulares])

In [93]:
def computa_reaj_retroativo(titulares):
    return sum([t.get_valor_reaj_retroativo() for t in titulares])

In [94]:
def computa_reaj_faixa(titulares):
    return sum([t.get_valor_reaj_faixa() for t in titulares])

In [95]:
computa_total_mensalidades(titulares)
#ok -> verificado

69128.36999999998

In [96]:
computa_reaj_retroativo(titulares)

0.0

In [97]:
computa_reaj_faixa(titulares)

206.18

In [98]:
computa_total_utilizacao(titulares)
#erro: deveria ser 8.703,71

7752.180000000002

In [99]:
def caso_especial_michel_osmar():
    for t in titulares:
        if t.nome == 'Michel Osmar Costa Paiva':
            mulher_do_michel = t.dependentes.pop(0)
            titulares.append(mulher_do_michel)
caso_especial_michel_osmar()

In [100]:
computa_total_utilizacao(titulares)
#computa_total_mensalidades(titulares)

7752.180000000002

In [101]:
from datetime import date
def generate_code(i):
    i = i + 1
    today = date.today()
    s = today.strftime("%d%m%Y")
    s = s + str(i)
    return s

In [102]:
lista_final = []
for i,t in enumerate(titulares):
    v = f'{t.get_valor_total():.2f}'.replace('.',',')
    c = generate_code(i)
    l = [t.nome.upper(), v , c]
    lista_final.append(l)

In [103]:
lista_final

[['CHRISTIANE SOUZA DE MACEDO BARBOSA', '958,38', '250820211'],
 ['MELQUISEDEQUE DA CONCEICAO LIMA', '767,36', '250820212'],
 ['JAQUELINE AIDA FERRETE', '749,94', '250820213'],
 ['ANELIZE IRENE APPELT', '1180,75', '250820214'],
 ['ALINE COSTA ELHKE', '1592,41', '250820215'],
 ['AREMILSON ELIAS DE OLIVEIRA', '857,48', '250820216'],
 ['CLAUDINEI DE OLIVEIRA PINHO', '2010,80', '250820217'],
 ['CLAUDIA APARECIDA PRATES DOS SANTOS', '1734,68', '250820218'],
 ['MARIA APARECIDA BOAVENTURA', '980,50', '250820219'],
 ['DOUGLAS LEGRAMANTE', '1051,16', '2508202110'],
 ['CARLOS PEREIRA SOARES', '448,26', '2508202111'],
 ['JULIO CEZAR MOZER SODRE', '2512,60', '2508202112'],
 ['ANTONIO SERGIO FLORINDO DOS SANTOS', '1712,21', '2508202113'],
 ['MARIA CONSUELO MOREIRA', '1250,57', '2508202114'],
 ['MARIA DE ARAUJO DAL MORO', '899,88', '2508202115'],
 ['LIRIAN KELI DOS SANTOS', '1306,43', '2508202116'],
 ['PEDRO VARGAS GROEFF', '2216,10', '2508202117'],
 ['EZEQUIEL FERREIRA BARBOSA', '1567,06', '2508202

In [113]:
#lista_final.append(["CLAUDEMIR MIRANDA BARBOZA", '74,43', '1104202199'])
#94 de utilização e 3 do boleto

In [114]:
#lista_final = lista_final[:-1]
#lista_final

In [104]:
import json
with open('data-agosto-21.json', 'w') as f:
    json.dump(lista_final, f)

In [105]:
titulares[0].report_values()

['Christiane Souza De Macedo Barbosa: 389.24',
 'Utilização: 0.00',
 'Reajuste retroativo: 0.00',
 'Reajuste faixa etária: 0.00',
 'Max Pederiva Barbosa: 389.24',
 'Reajuste retroativo: 0.00',
 'Reajuste faixa etária: 0.00',
 'Igor Souza Pederiva: 176.97',
 'Reajuste retroativo: 0.00',
 'Reajuste faixa etária: 0.00']

In [106]:
len(titulares)

75

In [107]:
len(lista_final)

75

In [108]:
sorted(lista_final, key=lambda x: x[0])

[['ADRIANA BARBOSA COELHO', '1374,53', '2508202157'],
 ['AGNALDO CORREA DE ALMEIDA', '1621,77', '2508202131'],
 ['ALEX TAVARES DE ALMEIDA', '1952,01', '2508202127'],
 ['ALINE ALVES DE MORAES', '1225,35', '2508202125'],
 ['ALINE COSTA ELHKE', '1592,41', '250820215'],
 ['ALINE SANTIAGO', '927,41', '2508202138'],
 ['ALVINO MORAES DE AMORIM', '2001,58', '2508202121'],
 ['ANA LUISA MATHIAS COSTA', '179,90', '2508202175'],
 ['ANA PAULA SCHIMIDT QUADROS', '786,84', '2508202120'],
 ['ANDRE BONIFACIO BALLASCHK', '341,39', '2508202156'],
 ['ANELIZE IRENE APPELT', '1180,75', '250820214'],
 ['ANTONIO SERGIO FLORINDO DOS SANTOS', '1712,21', '2508202113'],
 ['AREMILSON ELIAS DE OLIVEIRA', '857,48', '250820216'],
 ['ARIANE ZAMBON MIRANDA', '457,14', '2508202159'],
 ['AUREA DAYSE COSMO DA SILVA', '445,20', '2508202172'],
 ['CARLOS PEREIRA SOARES', '448,26', '2508202111'],
 ['CHRISTIANE SOUZA DE MACEDO BARBOSA', '958,38', '250820211'],
 ['CINTHIA ALVES PEREIRA MATTOS', '1048,38', '2508202124'],
 ['CLAR

In [120]:
nomes = [t.nome.upper().replace(' ', '-') for t in titulares]

In [121]:
#nomes

In [122]:
import os

In [123]:
#verifica se todos os boletos foram gerados
#arquivos = os.listdir('/home/juliano/Downloads/boletos-fev-21/')
#arquivos = [arq[:-4] for arq in arquivos if arq.endswith('.pdf')]
#set1 = set(arquivos)
#set2 = set(nomes)
#set2-set1

In [124]:
for t in titulares:
    if t.nome.startswith('Laura'):
        print(t.report_values())

['Laura De Paula Leite Weiss: 389.24', 'Utilização: 84.93', 'Reajuste retroativo: 18.57', 'Reajuste faixa etária: 0.00', 'Luciano Borba Weiss: 389.24', 'Reajuste retroativo: 16.15', 'Reajuste faixa etária: 0.00', 'Daniel Weiss: 176.97', 'Reajuste retroativo: 8.44', 'Reajuste faixa etária: 0.00', 'Vinicius Weiss: 176.97', 'Reajuste retroativo: 8.44', 'Reajuste faixa etária: 0.00', 'Arthur Weiss: 176.97', 'Reajuste retroativo: 8.44', 'Reajuste faixa etária: 0.00']


In [109]:
import pickle
pickle.dump(titulares, open("titulares-agosto-21", "wb" ) )